In [1]:
import pandas as pd
pd.DataFrame({'a': [1,2,3]})

,a
0,1
1,2
2,3


## sample function to extract data from kraken

In [2]:
import slumber
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from decimal import Decimal as D
import logging
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
import copy

import numpy as np

In [3]:
class Exchange:
  def __init__(self, url, commissions):
    self.url = url
    self.commissions = commissions
    self.api = None
    self.session = None

  def get_requests_session(self):
    if self.session is not None:
      return self.session

    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    self.session = s
    return s

  def get_slumber_api(self, append_slash=True):
    if self.api is not None:
      return self.api

    s = self.get_requests_session()
    self.api = slumber.API(self.url, session=s, append_slash=append_slash)
    return self.api

  def fetch(self, ticker):
    raise ValueError("Not implemented")

  def postprocess(self, ob1, ticker):
    raise ValueError("Not implemented")



class Kraken(Exchange):
  def __init__(self):
    super().__init__(
      url="https://api.kraken.com/0/public/",
      commissions = D('0.14')/100
    )

  def fetch(self, ticker):
    logging.debug('fetch kraken')

    api = self.get_slumber_api(append_slash=False)
    ob1 = api.Depth.get(pair=ticker, count=30)
    return ob1

  def postprocess(self, ob1, ticker):
    """
    ob1 - output of self.fetch(ticker)
    """
    logging.debug('postprocess kraken')
    
    if ob1['error']:
        raise ValueError("Kraken error: %s"%(ob1['error']))

    ob2 = copy.deepcopy(ob1)
    ob2 = list(ob2['result'].values())[0]
    for k1 in ['asks', 'bids']:
        ob2[k1] = pd.DataFrame(ob2[k1]).rename(columns={0:'price', 1:'amount', 2:'timestamp'})
        # convert to Decimal
        for k2 in ['price', 'amount']:
            ob2[k1][k2] = [D(x) for x in ob2[k1][k2].values]

    # commissions
    ob2['asks']['price pre-commission'] =  ob2['asks']['price']
    ob2['bids']['price pre-commission'] =  ob2['bids']['price']
    ob2['asks']['price'] = ob2['asks']['price'] * (1+self.commissions)
    ob2['bids']['price'] = ob2['bids']['price'] * (1-self.commissions)

    # set side
    ob2['asks']['side'] = 'ask'
    ob2['bids']['side'] = 'bid'

    ob3 = pd.concat([ob2['asks'], ob2['bids']], axis=0).reset_index(drop=True)
    ob3['source'] = 'kraken.com'
    ob3['ticker'] = ticker

    return ob3

In [4]:
kraken = Kraken()
ticker = 'XBTUSD'
order_book = kraken.fetch(ticker)
order_book = kraken.postprocess(order_book, ticker)

In [5]:
order_book

,price,amount,timestamp,price pre-commission,side,source,ticker
0,6653.802300000,0.933,1529650112,6644.50000,ask,kraken.com,XBTUSD
1,6656.205660000,0.296,1529650108,6646.90000,ask,kraken.com,XBTUSD
2,6656.305800000,0.013,1529650105,6647.00000,ask,kraken.com,XBTUSD
3,6657.807900000,0.015,1529650111,6648.50000,ask,kraken.com,XBTUSD
4,6658.008180000,3.000,1529650071,6648.70000,ask,kraken.com,XBTUSD
5,6658.308600000,1.000,1529650095,6649.00000,ask,kraken.com,XBTUSD
6,6659.209860000,8.000,1529649894,6649.90000,ask,kraken.com,XBTUSD
7,6659.810700000,0.072,1529650107,6650.50000,ask,kraken.com,XBTUSD
8,6660.511680000,1.990,1529650113,6651.20000,ask,kraken.com,XBTUSD
9,6661.613220000,2.801,1529650113,6652.30000,ask,kraken.com,XBTUSD
